In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from binance.client import Client
from tensorflow import keras

# Conexión a la API de Binance
api_key = 'T4Bw573BSJCbaHscSo8jn37lE1SOoGsNircF8f7B061WIKBxNkP5nx68vvAev9uk'
api_secret = 'p7VN6HHKR6ZtGkfEeZC3FongJPkR2yr7AjPtPDC8IeaOkASdKZYCKyTuiIfGw57q'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "2 days ago UTC")

# Conversión de los datos a un dataframe de pandas
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
data = data[['timestamp', 'close']]
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data.set_index('timestamp', inplace=True)
data['close'] = pd.to_numeric(data['close'], errors='coerce')

# Normalización de los datos
max_value = data['close'].max()
min_value = data['close'].min()
data['close'] = (data['close'] - min_value) / (max_value - min_value)

# Procesamiento de los datos para la entrada del modelo
data['t-1'] = data['close'].shift(1)
data.dropna(inplace=True)

X = np.array(data[['close', 't-1']])
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Carga del modelo entrenado
model = keras.models.load_model('ruta/para/guardar/el/modelo/btc_price_prediction_model.h5')

# Predicción del precio de Bitcoin en tiempo real
while True:
    kline = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1HOUR)[-2]
    timestamp = kline[0]
    close_price = float(kline[4])

    data = data.append({'timestamp': pd.to_datetime(timestamp, unit='ms'), 'close': close_price}, ignore_index=True)
    data['close'] = (data['close'] - min_value) / (max_value - min_value)
    data['t-1'] = data['close'].shift(1)
    data.dropna(inplace=True)  # Eliminamos la primera fila que ya no es necesaria
    X = np.array(data[['close', 't-1']])
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    yhat = model.predict(X[-1].reshape(1, X[-1].shape[0], X[-1].shape[1]))
    predicted_price = yhat[0][0] * (max_value - min_value) + min_value

    print(f'Actual price: {close_price:.2f}, Predicted price: {predicted_price:.2f}')
    time.sleep(60*60)  # Esperamos una hora antes de realizar la siguiente predicción


2023-05-03 08:29:58.361777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 08:29:58.437848: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 08:29:58.439313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 08:29:59.370851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-03 08:30:01.170948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-03 08:30:01.171284: W tensorflow/core/common_r

1/1 [==============================] - 0s 413ms/step


2023-05-03 08:30:02.489411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 08:30:02.492467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 08:30:02.495500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Actual price: 28306.53, Predicted price: 27029.96


KeyboardInterrupt: 

* Este script de Python utiliza la API de Binance para obtener datos históricos de precios de Bitcoin y realizar una predicción en tiempo real del precio de Bitcoin utilizando un modelo de redes neuronales previamente entrenado.

En primer lugar, se importan las librerías necesarias: "time" para establecer un tiempo de espera antes de realizar la siguiente predicción, "numpy" y "pandas" para procesar los datos, "matplotlib" para visualizar los resultados y "tensorflow.keras" para cargar el modelo previamente entrenado.

A continuación, se establece la conexión a la API de Binance utilizando las claves de acceso proporcionadas. Se obtienen los datos históricos de precios de Bitcoin utilizando el método "get_historical_klines" de la clase "Client" de la API de Binance, especificando el par de divisas "BTCUSDT" y el intervalo de tiempo de 1 hora. Los datos se almacenan en la variable "klines".

Los datos se convierten a un dataframe de Pandas para facilitar su procesamiento y se eliminan las columnas que no son necesarias. La columna de tiempo se convierte a formato de fecha y hora y se establece como índice del dataframe. El precio de cierre se convierte a un número decimal utilizando el método "to_numeric" y se normaliza en un rango de 0 a 1 utilizando los valores máximos y mínimos.

Se procesan los datos para la entrada del modelo, añadiendo una columna con el precio de cierre en el instante anterior y eliminando las filas con valores faltantes. Los datos se convierten a un array NumPy y se remodelan para tener la forma (número de muestras, número de pasos de tiempo, número de características).

A continuación, se carga el modelo previamente entrenado utilizando la función "keras.models.load_model". El modelo se guarda en un archivo con extensión ".h5" y se especifica la ruta donde se encuentra el archivo.

Finalmente, se utiliza un bucle while True para realizar la predicción del precio de Bitcoin en tiempo real. En cada iteración del bucle, se obtiene el precio de cierre más reciente utilizando el método "get_klines" de la API de Binance. Se añade la nueva muestra al dataframe de datos, se normaliza el precio de cierre utilizando los valores máximos y mínimos de los datos de entrenamiento, se procesan los datos para la entrada del modelo y se realiza la predicción utilizando el método "predict" del modelo cargado. Se desnormaliza el valor de la predicción y se muestra junto con el valor actual del precio de cierre. Se espera una hora antes de realizar la siguiente predicción utilizando la función "time.sleep".